In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import re

data_folder = 'data/'

In [2]:
API_KEY = ""

with open('api_key.txt') as f:
    API_KEY = f.read()

HEADERS = {"X-Api-Key": API_KEY}

In [27]:
def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder(data_folder)
create_folder(data_folder + 'commitees')
create_folder(data_folder + 'senate_members')
create_folder(data_folder + 'votes')
create_folder(data_folder + 'lobby')
create_folder(data_folder + 'bills')

In [8]:
# Request config
req_str = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset={offset}"

res = requests.get(req_str.format(offset=460), headers = HEADERS)

print(json.dumps(res.json(), indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2018 Pro Publica Inc. All Rights Reserved.",
    "results": {
        "chamber": "Senate",
        "offset": 460,
        "num_results": 20,
        "votes": [
            {
                "congress": 115,
                "chamber": "Senate",
                "session": 1,
                "roll_call": 117,
                "source": "https://www.senate.gov/legislative/LIS/roll_call_votes/vote1151/vote_115_1_00117.xml",
                "url": "https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=115&session=1&vote=00117",
                "vote_uri": "https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/117.json",
                "bill": {},
                "amendment": {},
                "nomination": {
                    "nomination_id": "PN47-115",
                    "number": "PN47",
                    "name": "Jay Clayton",
                    "agency": "Securities and Exchange C

# Fetching bills

In [7]:
print(API_KEY)

OoD35vcgzdv2xXYUA3TwEGRY6yt1UN24W4nsIhU9


In [20]:
bill_str = "https://api.propublica.org/congress/v1/{congress}/senate/bills/introduced.json?offset={offset}"

bill_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = bill_str.format(congress=115, offset = request_offset), headers = HEADERS)
    if('results' in res.json()):
        
        print(request_offset, end='\r')
        #print(json.dumps(res.json(), indent = 4))
        data_available = int(res.json()['results'][0]['num_results']) > 0

        if data_available:
            bills = res.json()['results'][0]['bills']
            bill_list.append(pd.io.json.json_normalize(bills, record_prefix=True))
    else:
        print(str(request_offset) + ' - Error: ' + res.json()['error'])
    request_offset += 20
    

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
4

In [28]:
df = pd.concat(bill_list,sort=True)
df.to_csv(data_folder + "bills/bills.csv")

Index(['active', 'bill_id', 'bill_slug', 'bill_type', 'bill_uri',
       'committee_codes', 'committees', 'congressdotgov_url', 'cosponsors',
       'cosponsors_by_party.D', 'cosponsors_by_party.I',
       'cosponsors_by_party.R', 'enacted', 'govtrack_url', 'gpo_pdf_uri',
       'house_passage', 'introduced_date', 'last_vote', 'latest_major_action',
       'latest_major_action_date', 'number', 'primary_subject',
       'senate_passage', 'short_title', 'sponsor_id', 'sponsor_name',
       'sponsor_party', 'sponsor_state', 'sponsor_title', 'sponsor_uri',
       'subcommittee_codes', 'summary', 'summary_short', 'title', 'vetoed'],
      dtype='object')

In [86]:
active_bills = df[(df['active']==1)]
active_bills.to_csv(data_folder + "bills/active_bills.csv")

# Fetching co-sponsors for each bill

,active,bill_id,bill_slug,bill_type,bill_uri,committee_codes,committees,congressdotgov_url,cosponsors,cosponsors_by_party.D,...,sponsor_name,sponsor_party,sponsor_state,sponsor_title,sponsor_uri,subcommittee_codes,summary,summary_short,title,vetoed
1,True,sconres55-115,sconres55,sconres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,NaN,...,Roy Blunt,R,MO,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A concurrent resolution authorizing the use of...,None
2,True,sconres56-115,sconres56,sconres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,NaN,...,Roy Blunt,R,MO,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A concurrent resolution providing for the use ...,None
7,True,sres712-115,sres712,sres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,1.0,...,Roy Blunt,R,MO,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A resolution authorizing the use of official o...,None
8,True,sconres54-115,sconres54,sconres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,1.0,...,Roy Blunt,R,MO,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A concurrent resolution authorizing the printi...,None
16,True,sres710-115,sres710,sres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,1.0,...,Susan Collins,R,ME,Sen.,https://api.propublica.org/congress/v1/members...,[],,,"A resolution supporting Lights On Afterschool,...",None
12,True,sres709-115,sres709,sres,https://api.propublica.org/congress/v1/115/bil...,[SSFR],Senate Foreign Relations Committee,https://www.congress.gov/bill/115th-congress/s...,55,22.0,...,Ron Johnson,R,WI,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A resolution condemning Russia's provocative a...,None
17,True,s3661-115,s3661,s,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,5,3.0,...,James M. Inhofe,R,OK,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A bill to provide for a program of the Departm...,None
1,True,sres704-115,sres704,sres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,27,15.0,...,Roy Blunt,R,MO,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A resolution expressing support for the goals ...,None
2,True,sres706-115,sres706,sres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,1.0,...,Mitch McConnell,R,KY,Sen.,https://api.propublica.org/congress/v1/members...,[],,,A resolution to authorize representation by th...,None
3,True,sres705-115,sres705,sres,https://api.propublica.org/congress/v1/115/bil...,[],,https://www.congress.gov/bill/115th-congress/s...,1,1.0,...,Johnny Isakson,R,GA,Sen.,https://api.propublica.org/congress/v1/members...,[],,,"A resolution designating December 3, 2018, as ...",None


In [154]:
def cosponsors_for_bill(bill_id, congress=115):
    req_url = "https://api.propublica.org/congress/v1/{congress}/bills/{bill_id}/cosponsors.json"
    res = requests.get(req_url.format(bill_id=bill_id, congress=congress), headers = HEADERS)
    co_sponsors = res.json()['results'][0]['cosponsors']
    print("Getting cosponsors for bill " + str(bill_id), end="\r")
    return co_sponsors

In [155]:
def extract_cosponsors(cosponsor_list):
    return [x['cosponsor_id'] for x in cosponsor_list]

In [156]:
tmp = active_bills.copy()

tmp['cosponsors_id'] = tmp.apply(lambda row: extract_cosponsors(cosponsors_for_bill(row['bill_slug'])), axis=1)



In [202]:
tmp.to_pickle(data_folder + "/bills/bills_and_sponsors.pickle")

In [229]:
tmp = pd.read_pickle(data_folder + "bills/bills_and_sponsors.pickle")
tmp = tmp.apply(lambda row: pd.Series([row['bill_id'], row['cosponsors_id'] + [row['sponsor_id']]], index=['a','b']), axis=1)
tmp = tmp.set_index('a')
tmp.head()

,b
a,
sconres55-115,[B000575]
sconres56-115,[B000575]
sres712-115,"[K000367, B000575]"
sconres54-115,"[K000367, B000575]"
sres710-115,"[S001203, C001035]"


In [233]:
df = tmp['b'].str.join('|').str.get_dummies()
df = df.transpose()
df.to_pickle(data_folder + "member_matrices/member_to_bill_sponsoring.pickle")

In [146]:
df_cosponsors

1           []
2           []
7    [K000367]
dtype: object

# Fetching senate members (congress 80 to 115)

In [ ]:
u = "https://api.propublica.org/congress/v1/{congress}/senate/members.json"

for i in range(80, 115 + 1):
    results = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results.json()['results'][0]['members'])
    df.to_csv("data/senate_members/senate_members_{congress}.csv".format(congress=i))

In [ ]:
df.head()

# Fetching vote data from senate

In [ ]:
# Request config
u = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset={offset}"

votes_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = u.format(offset = request_offset), headers = HEADERS)
    if('results' in res.json()):
        data_available = int(res.json()['results']['num_results']) > 0

        if data_available:
            votes = res.json()['results']['votes']
            votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
    else:
        print(str(request_offset) + ' - Error: ' + res.json()['error'])
    request_offset += 20

In [ ]:
df = pd.concat(votes_list,sort=True)
df.to_csv(data_folder + "votes/votes.csv")

In [ ]:
df.head()

# Fetching Commitees from senate

In [ ]:
# Request config
u = "https://api.propublica.org/congress/v1/{congress}/senate/committees.json"

for i in range(114, 115 + 1):
    create_folder(data_folder + 'commitees/commitees_' + str(i))
    results_commitee = requests.get(url = u.format(congress=i), headers=HEADERS)
    df = pd.io.json.json_normalize(results_commitee.json()['results'][0]['committees'])
    df_list = []
    for commitee_id in df['id']:
        subcommitee_id = df[df['id'] == commitee_id]['subcommittees']

        for subcommitee in subcommitee_id.values[0]:
            results_sub = requests.get(url = subcommitee['api_uri'], headers=HEADERS)
            if 'results' in results_sub.json():
                df_sub = pd.io.json.json_normalize(results_sub.json()['results'][0]['current_members'])
                df_sub['subcomitee'] = subcommitee['id']
                df_sub['comitee'] = commitee_id
                df_list.append(df_sub)

    if len(df_list) > 0:
        df_merged = pd.concat(df_list)
        df_merged.to_csv(data_folder + 'commitees/commitees_{congress}/members.csv'.format(congress = i,commitee_id = elem))

# Fetching Lobbying from senate

In [ ]:
re.compile("\((.*)\"([\w ]+)\"(.*)\)")

In [ ]:
# Request config
u = "https://api.propublica.org/congress/v1/lobbying/latest.json?offset={offset}"

votes_list = []
data_available = True
request_offset = 0

while data_available :
    res = requests.get(url = u.format(offset = request_offset), headers = HEADERS)
    j  = res.text.replace("\\\"","").replace("\\","").replace("\")", ")").replace("(\"", "(")

    try:
        res = json.loads(j)       
        if('results' in res):
            #print(request_offset)
            data_available = int(res['results'][0]['num_results']) > 0

            if data_available:
                votes = res['results'][0]['lobbying_representations']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
        else:
            print(str(request_offset) + ' - Error: ' + res['error'])
    except:
        print(str(request_offset) + ' - Error: Json File badly encoded')
    request_offset += 20

In [ ]:
df = pd.concat(votes_list,sort=True)
df.to_csv(data_folder + "lobby/lobby.csv")

In [ ]:
df.head()

# Getting voting position by member

In [ ]:
raw_senators = pd.DataFrame()

for i in range (115, 80 - 1, -1):
    df = pd.read_csv("data/senate_members/senate_members_{congress}.csv".format(congress = i))
    df['congress'] = i
    raw_senators = pd.concat([raw_senators, df], sort=False)
    
senators_id = raw_senators['id'].unique()

In [ ]:
u = "https://api.propublica.org/congress/v1/members/{member_id}/votes.json?offset={offset}"

import os
import json

completed_ids = os.listdir("data/votes")

for senator_id in senators_id[2::4]:

    if "votes_{id}.csv".format(id=senator_id) in completed_ids:
        continue
        
    error_raised = False
    votes_list = []
    data_available = True
    request_offset = 0

    while data_available :
        res = requests.get(url = u.format(member_id = senator_id, offset = request_offset), headers = HEADERS)
        jObj = json.loads(res.text.replace('\n', ' '))
                
        #print(res.text)
        if(res.status_code == 200):
            data_available = int(jObj['results'][0]['num_results']) > 0

            if data_available:
                print(str(senator_id) + " offset: " + str(request_offset), end='\r')
                votes = jObj['results'][0]['votes']
                votes_list.append(pd.io.json.json_normalize(votes, record_prefix=True))
                
        else:
            print(str(res.status_code))
            error_raised = True
            break
        
        request_offset += 20
        
    if not error_raised and len(votes_list) > 0:
        df = pd.concat(votes_list,sort=False)
        df.to_csv(data_folder + "votes/votes_{id}.csv".format(id=senator_id), index=False)
